In [1]:
import os
from datetime import datetime
import pyspark.sql.functions as F
from gcpspark import *

Dotenv installed
Loaded variables .env True
Pyspark installed
JAVA:  /usr/lib/jvm/temurin-8-jdk-amd64
DATAPROC:  True
Current GCP Project Name: cencosudx
Current ENVIRONMENT: staging


In [2]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
ENV = os.environ["ENVIRONMENT"]

In [4]:
dataset_id = 'data_analytics_default'
table_id = 'BQ_CO_SM_UMV_01'

stock_table_id = 'BQ_CO_SM_STOCK_01'
master_active_table_id = 'BQ_CO_SM_ACTIVE_01'


In [ ]:
spark = create_pyspark(name="col-stock-umv",connection="gcp")

In [6]:
umv_df = (
    spark.read.format("bigquery")
    .option("table", f"{dataset_id}.{table_id}")
    .load()
)
umv_df = umv_df.drop("_PARTITIONTIME", "_PARTITIONDATE")
umv_df = umv_df.distinct()
umv_df.printSchema()

root
 |-- LOCAL_ID: string (nullable = true)
 |-- ITEM_ID: string (nullable = true)
 |-- EAN: string (nullable = true)
 |-- DATE: date (nullable = true)
 |-- DAY: string (nullable = true)
 |-- WEEK_DAY: long (nullable = true)
 |-- FORECAST: long (nullable = true)
 |-- UMV: long (nullable = true)
 |-- MAPE: long (nullable = true)
 |-- MODEL: string (nullable = true)



In [7]:
print(display(umv_df.groupby("DATE").count().orderBy(F.col("DATE").desc())))

         DATE  count
0  2023-11-26   6237
1  2023-11-25   7364
2  2023-11-24   4540
3  2023-11-23   4576
4  2023-11-22   4796
5  2023-11-21   4957
6  2023-11-20   5826
7  2023-11-19   2598
8  2023-11-18   2978
9  2023-11-17   1462


In [19]:
print(display(umv_df,5))

  LOCAL_ID  ITEM_ID   EAN        DATE     DAY  WEEK_DAY  FORECAST  UMV  MAPE    MODEL
0       76  0011567   116  2023-11-21  MARTES         3        54   38    25  PROPHET
1       42  0011567   116  2023-11-21  MARTES         3        44   28    26  SARIMAX
2      191  0002670   659  2023-11-21  MARTES         3        41   26    24  PROPHET
3       23  0248978  1991  2023-11-21  MARTES         3         7    1    16  SARIMAX
4       87  0752396  3940  2023-11-21  MARTES         3        79   70    15    ARIMA


In [8]:
stock_df = (
    spark.read.format("bigquery")
    .option("table", f"{dataset_id}.{stock_table_id}")
    .load()
)
stock_df = stock_df.distinct()
stock_df = stock_df.drop("_PARTITIONTIME", "_PARTITIONDATE","fecha_stock")
stock_df.printSchema()

root
 |-- tienda_stock: string (nullable = true)
 |-- sku_stock: string (nullable = true)
 |-- ean_stock: string (nullable = true)
 |-- unidades_stock: double (nullable = true)



In [9]:
print(display(stock_df,5))

  tienda_stock sku_stock      ean_stock  unidades_stock
0          105   3417356  7702189056160            54.0
1          248   3578854  7702004111142            40.0
2          249   0296274  7702425524828            64.0
3          249   3453039  7702535028186            36.0
4           56   1666085   645667183583          2000.0


In [10]:
active_df = (
    spark.read.format("bigquery")
    .option("table", f"{dataset_id}.{master_active_table_id}")
    .load()
)
active_df = active_df.distinct()
active_df = active_df.drop("_PARTITIONTIME", "_PARTITIONDATE")
active_df.printSchema()

root
 |-- sku_stock: string (nullable = true)
 |-- ean_stock: string (nullable = true)
 |-- tienda_stock: string (nullable = true)



In [11]:
print(display(active_df,5))

  sku_stock      ean_stock tienda_stock
0   0000019        2400011           85
1   0000102  7702277500025           24
2   0000251  7701018007519           45
3   0000541             86           38
4   0001792  7702450022214           13


# Filter items with no stock or lower than umv

In [12]:
print("#stocks filtered: ",stock_df.count())
print("# umv: ",umv_df.count())

#stocks filtered:  2241452


# umv:  248639


In [13]:
display(umv_df.groupby("DATE").count())

,DATE,count
0,2023-11-22,4796
1,2023-11-25,7364
2,2023-11-17,1462
3,2023-11-18,2978
4,2023-10-12,4805
5,2023-11-24,4540
6,2023-11-04,6724
7,2023-10-29,5993
8,2023-10-13,5179
9,2023-11-05,6224


In [14]:
# display(stock_df.groupby("fecha_stock").count())

In [15]:
today_date = datetime.today().date()
umv_df = umv_df.filter(F.col("DATE")==F.lit(today_date))
stock_filter = umv_df.join(stock_df,\
                           (umv_df.LOCAL_ID == stock_df.tienda_stock) &\
                           (umv_df.EAN == stock_df.ean_stock) &\
                           (umv_df.ITEM_ID == stock_df.sku_stock),\
                           "inner")
print("# filtered umv: ",stock_filter.count())

# filtered umv:  4081


In [16]:
stock_filter = stock_filter.drop("ean_stock","tienda_stock","sku_stock","fecha_stock")

In [17]:
stock_filter.printSchema()

root
 |-- LOCAL_ID: string (nullable = true)
 |-- ITEM_ID: string (nullable = true)
 |-- EAN: string (nullable = true)
 |-- DATE: date (nullable = true)
 |-- DAY: string (nullable = true)
 |-- WEEK_DAY: long (nullable = true)
 |-- FORECAST: long (nullable = true)
 |-- UMV: long (nullable = true)
 |-- MAPE: long (nullable = true)
 |-- MODEL: string (nullable = true)
 |-- unidades_stock: double (nullable = true)



In [18]:
print(display(stock_filter,5))

  LOCAL_ID  ITEM_ID            EAN        DATE     DAY  WEEK_DAY  FORECAST   UMV  MAPE     MODEL  unidades_stock
0       13  3499916  7703616031606  2023-11-21  MARTES         3      1032  1024    11     ARIMA         34136.0
1       13  3475154  7705326002303  2023-11-21  MARTES         3        17     1    30  ENSEMBLE           120.0
2       15  3242338  7702001133611  2023-11-21  MARTES         3        14     1    30   SARIMAX           159.0
3       15  0277966  7702434214000  2023-11-21  MARTES         3        20     7    21  ENSEMBLE           300.0
4       22  0835476  7705326079367  2023-11-21  MARTES         3         7     1    28   SARIMAX           104.0


In [20]:
stock_filtered=stock_filter.withColumn("Diff", F.round(F.col("unidades_stock")-F.col("UMV"))).filter(F.col("Diff")>0)
print("# filtered umv: ",stock_filtered.count())

# filtered umv:  4015


In [21]:
# stock_filtered = stock_filtered.distinct()
# print("# filtered umv: ",stock_filtered.count())
print(display(stock_filtered))

  LOCAL_ID  ITEM_ID            EAN        DATE     DAY  WEEK_DAY  FORECAST   UMV  MAPE     MODEL  unidades_stock     Diff
0       13  3499916  7703616031606  2023-11-21  MARTES         3      1032  1024    11     ARIMA         34136.0  33112.0
1       13  3475154  7705326002303  2023-11-21  MARTES         3        17     1    30  ENSEMBLE           120.0    119.0
2       15  3242338  7702001133611  2023-11-21  MARTES         3        14     1    30   SARIMAX           159.0    158.0
3       15  0277966  7702434214000  2023-11-21  MARTES         3        20     7    21  ENSEMBLE           300.0    293.0
4       22  0835476  7705326079367  2023-11-21  MARTES         3         7     1    28   SARIMAX           104.0    103.0
5       24  0349839  7702192422051  2023-11-21  MARTES         3        11     2    14     ARIMA           129.0    127.0
6       33  0195645  7702137636864  2023-11-21  MARTES         3         5     1    22     HOLTW            73.0     72.0
7       34  3334320  770

# Filter items no active

In [ ]:
# print("#active: ",active_df.count())
# active_df = active_df.filter(F.col("estado_producto")=="S")
# active_df = active_df.distinct()
# print("#active: ",active_df.count())

In [22]:
stock_filter.printSchema()

root
 |-- LOCAL_ID: string (nullable = true)
 |-- ITEM_ID: string (nullable = true)
 |-- EAN: string (nullable = true)
 |-- DATE: date (nullable = true)
 |-- DAY: string (nullable = true)
 |-- WEEK_DAY: long (nullable = true)
 |-- FORECAST: long (nullable = true)
 |-- UMV: long (nullable = true)
 |-- MAPE: long (nullable = true)
 |-- MODEL: string (nullable = true)
 |-- unidades_stock: double (nullable = true)



In [23]:
active_filter = stock_filtered.join(active_df, (stock_filtered.LOCAL_ID ==active_df.tienda_stock) & (stock_filtered.EAN ==active_df.ean_stock)& (stock_filtered.ITEM_ID ==active_df.sku_stock), "inner")
print("# filtered umv: ",active_filter.count())

# filtered umv:  4009


In [26]:
active_filter = active_filter.drop("sku_stock","ean_stock","tienda_stock")
active_filter = active_filter.distinct()
print("# filtered umv: ",active_filter.count())
active_filter.printSchema()

# filtered umv:  4009
root
 |-- LOCAL_ID: string (nullable = true)
 |-- ITEM_ID: string (nullable = true)
 |-- EAN: string (nullable = true)
 |-- DATE: date (nullable = true)
 |-- DAY: string (nullable = true)
 |-- WEEK_DAY: long (nullable = true)
 |-- FORECAST: long (nullable = true)
 |-- UMV: long (nullable = true)
 |-- MAPE: long (nullable = true)
 |-- MODEL: string (nullable = true)
 |-- unidades_stock: double (nullable = true)
 |-- Diff: double (nullable = true)



In [27]:
print(display(active_filter,5))

  LOCAL_ID  ITEM_ID            EAN        DATE     DAY  WEEK_DAY  FORECAST  UMV  MAPE     MODEL  unidades_stock   Diff
0       72  0752396           3940  2023-11-21  MARTES         3        28   16    19     HOLTW          414.83  399.0
1        3  0835434  7705326079343  2023-11-21  MARTES         3        15    1    28     HOLTW          144.00  143.0
2       19  3522944  7702425014374  2023-11-21  MARTES         3         5    1    23  ENSEMBLE          150.00  149.0
3        3  0652745  7702175156874  2023-11-21  MARTES         3         8    1    30     ARIMA          225.00  224.0
4       73  0786481  7703616093604  2023-11-21  MARTES         3         6    1    27   SARIMAX          260.00  259.0


In [28]:
# active_filter.select("LOCAL_ID").distinct().count()

In [29]:
final_table=active_filter\
                .withColumnRenamed("MAPE","ERROR")\
                .withColumnRenamed("Diff","DIFF")\
                .withColumnRenamed("unidades_stock","STOCK")\
                .withColumn("STATE",F.lit("S"))\
                .withColumn("UN",F.lit("N/D"))

final_table.printSchema()
# final_table.limit(5).toPandas()             

root
 |-- LOCAL_ID: string (nullable = true)
 |-- ITEM_ID: string (nullable = true)
 |-- EAN: string (nullable = true)
 |-- DATE: date (nullable = true)
 |-- DAY: string (nullable = true)
 |-- WEEK_DAY: long (nullable = true)
 |-- FORECAST: long (nullable = true)
 |-- UMV: long (nullable = true)
 |-- ERROR: long (nullable = true)
 |-- MODEL: string (nullable = true)
 |-- STOCK: double (nullable = true)
 |-- DIFF: double (nullable = true)
 |-- STATE: string (nullable = false)
 |-- UN: string (nullable = false)



## REAL SALES

In [36]:
# Data collection
dataset_id = 'data_analytics_default'
table_id = 'BQ_CO_SM_SALES_02'

df = (
    spark.read.format("bigquery")
    .option("table", f"{dataset_id}.{table_id}")
    .load()
)

In [37]:
df = df.filter(F.col("DATE")==F.date_sub(F.current_date(), 1))

In [38]:
df = df.groupBy("EAN","ITEM_ID","LOCAL_ID").sum("UNITS")

In [39]:
df.printSchema(), df.count()

root
 |-- EAN: string (nullable = true)
 |-- ITEM_ID: string (nullable = true)
 |-- LOCAL_ID: string (nullable = true)
 |-- sum(UNITS): double (nullable = true)



(None, 278341)

In [40]:
display(df)

,EAN,ITEM_ID,LOCAL_ID,sum(UNITS)
0,55,0003087,16,0.135
1,86,0000541,49,1.125
2,161,0000095,11,9.930
3,178,0000206,84,1.260
4,208,0003088,73,1.565
5,291,0002965,81,11.415
6,468,0002647,44,8.735
7,505,0003271,70,2.920
8,611,0037603,28,0.460
9,628,0002812,15,1.860


In [41]:
final_table_real_sales = final_table.join(df,on=["EAN", "ITEM_ID", "LOCAL_ID"], how="inner").withColumnRenamed("sum(UNITS)","REAL_SALE")
final_table_real_sales = final_table_real_sales.select("LOCAL_ID","ITEM_ID","EAN","DATE",\
                                 "DAY","WEEK_DAY","FORECAST","UMV",\
                                 "ERROR","MODEL","REAL_SALE","DIFF",\
                                 "STOCK","STATE")

In [42]:
final_table_real_sales = final_table_real_sales.filter(F.col("REAL_SALE")<=F.col("UMV"))

In [43]:
final_table_real_sales.printSchema()
final_table_real_sales=final_table_real_sales.cache()

root
 |-- LOCAL_ID: string (nullable = true)
 |-- ITEM_ID: string (nullable = true)
 |-- EAN: string (nullable = true)
 |-- DATE: date (nullable = true)
 |-- DAY: string (nullable = true)
 |-- WEEK_DAY: long (nullable = true)
 |-- FORECAST: long (nullable = true)
 |-- UMV: long (nullable = true)
 |-- ERROR: long (nullable = true)
 |-- MODEL: string (nullable = true)
 |-- REAL_SALE: double (nullable = true)
 |-- DIFF: double (nullable = true)
 |-- STOCK: double (nullable = true)
 |-- STATE: string (nullable = false)



In [44]:
final_table_real_sales.count()

202

In [45]:
display(final_table_real_sales)

,LOCAL_ID,ITEM_ID,EAN,DATE,DAY,WEEK_DAY,FORECAST,UMV,ERROR,MODEL,REAL_SALE,DIFF,STOCK,STATE
0,103,0000254,1045,2023-11-21,MARTES,3,90,84,9,HOLTW,21.635,259.0,342.66,S
1,7,0400303,2700002000005,2023-11-21,MARTES,3,946,936,16,SARIMAX,604.680,21733.0,22668.53,S
2,145,0489326,2700008000009,2023-11-21,MARTES,3,2478,2467,17,SARIMAX,754.360,7061.0,9528.21,S
3,47,0489326,2700008000009,2023-11-21,MARTES,3,73,61,19,PROPHET,53.783,2135.0,2195.97,S
4,103,0003111,550,2023-11-21,MARTES,3,67,52,24,ENSEMBLE,23.565,524.0,575.70,S
5,103,3462899,7703616030579,2023-11-21,MARTES,3,22,11,17,ENSEMBLE,9.000,19.0,30.00,S
6,13,0400303,2700002000005,2023-11-21,MARTES,3,2237,2231,9,ARIMA,2054.301,1088.0,3319.47,S
7,136,0400303,2700002000005,2023-11-21,MARTES,3,1376,1372,6,ENSEMBLE,1338.887,4048.0,5420.11,S
8,40,0249768,24,2023-11-21,MARTES,3,85,66,30,HOLTW,52.595,1008.0,1074.18,S
9,43,3499916,7703616031606,2023-11-21,MARTES,3,355,349,9,PROPHET,287.000,11815.0,12164.00,S


## -----

## SCORE

In [ ]:
score = spark.read.parquet(f"gs://{ENV}-bucket-dataproc-bigquery/dataproc/tmp/co/sm/score_colombia")

In [ ]:
score.printSchema()

In [ ]:
score = score.withColumn("SCORE", F.col("SCORE").cast("double"))

In [ ]:
score = score.select(
    F.col("LOCAL_ID").alias("LOCAL_ID_df1"),
    F.col("ITEM_ID").alias("ITEM_ID_df1"),
    F.col("EAN").alias("EAN_df1"),
    F.col("SCORE")
    # Alias other columns as needed
)

In [ ]:
join_condition = (final_table_real_sales["EAN"] == score["EAN_df1"]) & (final_table_real_sales["ITEM_ID"] == score["ITEM_ID_df1"]) & (final_table_real_sales["LOCAL_ID"] == score["LOCAL_ID_df1"])

# Perform an inner join
final_table_real_sales_scored = final_table_real_sales.join(score, join_condition, "left").drop("LOCAL_ID_df1","ITEM_ID_df1","EAN_df1")

In [ ]:
final_table_real_sales_scored.printSchema()

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, DateType, LongType, DoubleType

def make_all_columns_nullable(input_df):
    # Get the original schema
    original_schema = input_df.schema

    # Create a new schema with all columns set to nullable
    new_fields = []
    for field in original_schema.fields:
        new_field = StructField(field.name, field.dataType, nullable=True)
        new_fields.append(new_field)

    new_schema = StructType(new_fields)

    # Create a new DataFrame with the updated schema
    nullable_df = input_df.withColumnRenamed("_PARTITIONTIME", "_PARTITIONTIME_str")  # Rename the column temporarily
    nullable_df = spark.createDataFrame(nullable_df.rdd, new_schema)
    nullable_df = nullable_df.withColumnRenamed("_PARTITIONTIME_str", "_PARTITIONTIME")  # Revert the column name if needed

    return nullable_df

In [ ]:
final_table_real_sales_scored = make_all_columns_nullable(final_table_real_sales_scored)

In [ ]:
final_table_real_sales_scored.count()

In [ ]:
final_table_real_sales_scored.write.format("bigquery") \
    .option("temporaryGcsBucket",f"{ENV}-bucket-dataproc-bigquery/dataproc/co_core") \
    .option("table", "data_analytics_default.BQ_CO_SM_RESULT_CORE_01") \
    .mode("append") \
    .save()

In [ ]:
final_table_real_sales_scored = spark.read.format("bigquery") \
        .option("table", "data_analytics_default.BQ_CO_SM_RESULT_CORE_01") \
        .load()

In [ ]:
print(display(final_table_real_sales_scored,5))

In [ ]:
print(display(final_table_real_sales_scored.groupBy("DATE","LOCAL_ID").count().orderBy(F.col("count").desc())))

In [ ]:
df3 = final_table_real_sales_scored.groupBy("DATE","LOCAL_ID").count().orderBy(F.col("count").desc())
df3.agg(F.avg(df3["count"])).collect()[0][0]

In [ ]:
print(display(final_table_real_sales_scored.groupBy("LOCAL_ID").count().orderBy(F.col("count").desc()),10))

In [ ]:
print(display(final_table_real_sales_scored.groupBy("LOCAL_ID").count().orderBy(F.col("count")),10))

In [ ]:
df4 = final_table_real_sales_scored.groupBy("LOCAL_ID").count().orderBy(F.col("count").desc())

In [ ]:
df4.agg(F.avg(df4["count"])).collect()[0][0]

In [ ]:
print(display(final_table_real_sales_scored.groupBy("DATE").count().orderBy(F.col("DATE").desc()),10))

In [ ]:
# spark.stop()